In [0]:
%sql
-- Create SQL Procedure for ADS_RDS_EVENT_STATUS_IDENTIF_STATE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK'
AS
BEGIN

truncate table gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_HUMANTASK;

insert into gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_HUMANTASK
SELECT 
    CIDLA, 
    HUMANTASK_ID, 
    CPPROP_VALUE_TEXT, 
    CPPT_CODE
FROM ( 
    SELECT 
        CIDLA, 
        HUMANTASK_ID, 
        ONTABLET,
        HEADERS, 
        SOURCESYSTEM
    FROM (
        SELECT 
            CIDLA, 
            get_json_object(JSONDATA, '$.humanTask.id') AS HUMANTASK_ID,  
            get_json_object(JSONDATA, '$.completeExtension.onTablet') AS ONTABLET, 
            get_json_object(JSONDATA, '$.completeExtension.headers') AS HEADERS, 
            NULL AS sourcesystem,
            ROW_NUMBER() OVER (
                PARTITION BY get_json_object(JSONDATA, '$.humanTask.id') 
                ORDER BY TIMESTAMP DESC
            ) AS RN
        FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full
        WHERE TYPE = 'HT_COMPLETED'
          AND SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE)
        
        UNION ALL 
        
        SELECT 
            cidla,
            get_json_object(jsondata, '$.humanTask.id') AS HUMANTASK_ID,
            NULL AS ONTABLET,
            NULL AS HEADERS,
            get_json_object(jsondata, '$.humanTask.sourceSystem') AS sourcesystem,
            ROW_NUMBER() OVER (
                PARTITION BY get_json_object(JSONDATA, '$.humanTask.id') 
                ORDER BY TIMESTAMP DESC
            ) AS RN
        FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full
        WHERE TYPE LIKE 'HT%' 
          AND type <> 'HT_RECONCILE'
          AND SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE)
    ) 
    WHERE RN = 1
)
UNPIVOT (
    CPPROP_VALUE_TEXT 
    FOR CPPT_CODE IN (
        ONTABLET AS SMA_ONTABLET,
        HEADERS AS SMA_HEADERS,
        SOURCESYSTEM AS SMA_SOURCESYSTEM
    )
);

truncate table gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_HUMANTASK;

INSERT INTO gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_HUMANTASK 
      SELECT
             CPPROP_KEY
        ,    CPPROP_SOURCE_ID
        ,    CPPROP_SOURCE_SYSTEM_ID
        ,    CPPROP_SOURCE_SYS_ORIGIN
        ,    CPPROP_PARENT_KEY                                         
        ,    CASEPH_KEY
        ,    CPPTP_KEY
        ,    CTP_KEY
        ,    CASE_KEY
        ,    CASE_START_DATE
        ,    CPPROP_VALUE_TEXT
        ,    CPPROP_VALUE_DATE
        ,    CPPROP_VALUE_NUMBER
        ,    CPPRV_KEY
        ,    CPPROP_DELETED_FLAG
      FROM (
      SELECT 
        CPPROP_KEY,
        CPPROP_SOURCE_ID,
        CPPROP_SOURCE_SYSTEM_ID,
        CPPROP_SOURCE_SYS_ORIGIN,
        CPPROP_PARENT_KEY,
        CASEPH_KEY, 
        CPPTP_KEY, 
        CTP_KEY,
        CASE_KEY,
        CASE_START_DATE,
        CPPROP_VALUE_TEXT, 
        CPPROP_VALUE_DATE,
        CPPROP_VALUE_NUMBER,
        CPPRV_KEY,
        CPPROP_DELETED_FLAG
      FROM (
                  SELECT 
                     null as CPPROP_KEY
                    , SRC.HUMANTASK_ID||'.'||SRC.CPPT_CODE AS CPPROP_SOURCE_ID
                    ,'SMA' AS CPPROP_SOURCE_SYSTEM_ID
                    ,'SMA_MONITOR_EVENTS' AS CPPROP_SOURCE_SYS_ORIGIN
                    ,-1 as CPPROP_PARENT_KEY
                    ,CPH.CASEPH_KEY
                    ,CPPTP.CPPTP_KEY
                    ,NVL(C.CTP_KEY,-1) AS CTP_KEY
                    ,NVL(C.CASE_KEY,-1) AS CASE_KEY
                    ,NVL(C.CASE_START_DATE,TO_DATE('01011000','DDMMYYYY')) AS CASE_START_DATE
                    ,SRC.CPPROP_VALUE_TEXT
                    ,NULL AS CPPROP_VALUE_DATE
                    ,NULL AS CPPROP_VALUE_NUMBER
                    ,-1 AS CPPRV_KEY
                    ,'N' AS CPPROP_DELETED_FLAG   
            from ( 
                  SELECT CIDLA, HUMANTASK_ID, CPPROP_VALUE_TEXT, CPPT_CODE
                    FROM gap_catalog.ads_etl_owner.STG_SMA_CASE_PHASE_PROPERTIES_HUMANTASK
                    ) SRC
                  JOIN gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES CPPTP
                    ON SRC.CPPT_CODE = CPPTP.CPPTP_SOURCE_ID
                    AND CPPTP.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
                    AND CPPTP.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
                    AND CPPTP.CPPTP_VALID_TO= DATE'3000-01-01'  
                  JOIN gap_catalog.ads_owner.CASE_PHASES CPH
                    ON CPH.CASEPH_SOURCE_ID=SRC.HUMANTASK_ID 
                    AND CPH.CPCAT_KEY=2 
                    AND CPH.CASEPH_SOURCE_SYSTEM_ID='SMA' 
                    AND CPH.CASEPH_SOURCE_SYS_ORIGIN='SMA_MONITOR_EVENTS' 
                    AND CPH.CASEPH_VALID_TO=DATE'3000-01-01'
                  LEFT JOIN gap_catalog.ads_owner.CASES C 
                    ON C.CASE_SOURCE_ID=SRC.CIDLA 
                    AND C.CASE_SOURCE_SYSTEM_ID='SMA' 
                    AND C.CASE_SOURCE_SYS_ORIGIN='SMA_MONITOR_EVENTS' 
                    AND C.CASE_VALID_TO=DATE'3000-01-01'
              )
      )      
;

DROP TABLE IF EXISTS gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF;

CREATE TABLE gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF
(
  tech_del_flg CHAR(1),
  tech_new_rec CHAR(1),
  tech_rid VARCHAR(255),
  CPPROP_KEY BIGINT,
  CPPROP_KEY_NEW BIGINT GENERATED ALWAYS AS IDENTITY (START WITH 1 INCREMENT BY 1),
  CPPROP_SOURCE_ID VARCHAR(400),
  CPPROP_SOURCE_SYSTEM_ID VARCHAR(120),
  CPPROP_SOURCE_SYS_ORIGIN VARCHAR(120),
  CPPROP_PARENT_KEY BIGINT,
  CASEPH_KEY BIGINT,
  CPPTP_KEY BIGINT,
  CTP_KEY BIGINT,
  CASE_KEY BIGINT,
  CASE_START_DATE DATE,
  CPPROP_VALUE_TEXT VARCHAR(4000),
  CPPROP_VALUE_DATE DATE,
  CPPROP_VALUE_NUMBER DECIMAL,
  CPPRV_KEY BIGINT
);

insert into gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF
(
  tech_del_flg,
  tech_new_rec,
  tech_rid,
  CPPROP_KEY,
  CPPROP_SOURCE_ID,
  CPPROP_SOURCE_SYSTEM_ID,
  CPPROP_SOURCE_SYS_ORIGIN,
  CPPROP_PARENT_KEY,
  CASEPH_KEY,
  CPPTP_KEY,
  CTP_KEY,
  CASE_KEY,
  CASE_START_DATE,
  CPPROP_VALUE_TEXT,
  CPPROP_VALUE_DATE,
  CPPROP_VALUE_NUMBER,
  CPPRV_KEY
)
select  
   src.CPPROP_DELETED_FLAG as tech_del_flg, 
   case when trg.CPPROP_SOURCE_ID is null then 'Y' else 'N' end as tech_new_rec,
   trg.rid as tech_rid,
   trg.CPPROP_KEY,
   src.CPPROP_SOURCE_ID, 
   src.CPPROP_SOURCE_SYSTEM_ID, 
   src.CPPROP_SOURCE_SYS_ORIGIN, 
   src.CPPROP_PARENT_KEY, 
   src.CASEPH_KEY, 
   src.CPPTP_KEY, 
   src.CTP_KEY, 
   src.CASE_KEY, 
   src.CASE_START_DATE, 
   src.CPPROP_VALUE_TEXT, 
   src.CPPROP_VALUE_DATE, 
   src.CPPROP_VALUE_NUMBER, 
   src.CPPRV_KEY
 from 
    (select CPPROP_SOURCE_ID, 
       CPPROP_SOURCE_SYSTEM_ID, 
       CPPROP_SOURCE_SYS_ORIGIN, 
       CPPROP_PARENT_KEY, 
       CASEPH_KEY, 
       CPPTP_KEY, 
       CTP_KEY, 
       CASE_KEY, 
       CASE_START_DATE, 
       CPPROP_VALUE_TEXT, 
       CPPROP_VALUE_DATE, 
       CPPROP_VALUE_NUMBER, 
       CPPROP_DELETED_FLAG, 
       CPPRV_KEY
       from gap_catalog.ads_etl_owner.XC_SMA_CASE_PHASE_PROPERTIES_HUMANTASK xc
   ) src LEFT JOIN
    (select  cpprop_key||'.'||cpprop_valid_from||'.'||t.ctp_key||'.'||case_start_date as rid, t.* from gap_catalog.ads_owner.CASE_PHASE_PROPERTIES t
    JOIN gap_catalog.ads_owner.CASE_PHASE_PROPERTY_TYPES CPPTP
        ON  CPPTP.CPPTP_SOURCE_SYSTEM_ID = 'RDS'
                    AND CPPTP.CPPTP_SOURCE_SYS_ORIGIN = 'RDS_ANALYTICALCPPROPERTYTYPES'
                    AND CPPTP.CPPTP_VALID_TO= DATE'3000-01-01' 
                    AND CPPTP.CPPTP_KEY = t.CPPTP_KEY
      where CPPROP_CURRENT_FLAG  = 'Y'
        and CPPROP_VALID_TO  = to_date('01013000','ddMMyyyy')
            ) trg
ON trg.CPPROP_SOURCE_ID = src.CPPROP_SOURCE_ID
 and trg.CPPROP_SOURCE_SYSTEM_ID = src.CPPROP_SOURCE_SYSTEM_ID
 and trg.CPPROP_SOURCE_SYS_ORIGIN = src.CPPROP_SOURCE_SYS_ORIGIN
 and trg.CPPTP_KEY = src.CPPTP_KEY
 and trg.CTP_KEY = src.CTP_KEY
 and trg.CASE_START_DATE = src.CASE_START_DATE
 and trg.CPPROP_VALID_TO = to_date('30000101','yyyyMMdd') WHERE (
     decode( src.CPPROP_PARENT_KEY,trg.CPPROP_PARENT_KEY,1,0 ) = 0  or
     decode( src.CASEPH_KEY,trg.CASEPH_KEY,1,0 ) = 0  or
     decode( src.CASE_KEY,trg.CASE_KEY,1,0 ) = 0  or
     decode( src.CPPROP_VALUE_TEXT,trg.CPPROP_VALUE_TEXT,1,0 ) = 0  or
     decode( src.CPPROP_VALUE_DATE,trg.CPPROP_VALUE_DATE,1,0 ) = 0  or
     decode( src.CPPROP_VALUE_NUMBER,trg.CPPROP_VALUE_NUMBER,1,0 ) = 0  or
     decode( src.CPPROP_DELETED_FLAG,trg.CPPROP_DELETED_FLAG,1,0 ) = 0  or
     decode( src.CPPRV_KEY,trg.CPPRV_KEY,1,0 ) = 0 or 
     trg.CPPROP_SOURCE_ID is null   );

update  gap_catalog.ads_owner.CASE_PHASE_PROPERTIES set
       CPPROP_UPDATED_DATETIME = CURRENT_TIMESTAMP(),
       CPPROP_UPDATE_PROCESS_KEY = p_process_key,
       CPPROP_CURRENT_FLAG = 'N', 
       CPPROP_VALID_TO = to_date(p_load_date,'yyyy-MM-dd')-1
 where CPPROP_CURRENT_FLAG = 'Y'
   and CPPROP_VALID_TO = to_date('30000101','yyyyMMdd')
   and (cpprop_key||'.'||cpprop_valid_from||'.'||ctp_key||'.'||case_start_date) in (select tech_rid from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF where tech_rid is not null);

  insert into gap_catalog.ads_owner.CASE_PHASE_PROPERTIES 
  ( CPPROP_KEY, 
    CPPROP_SOURCE_ID, 
    CPPROP_SOURCE_SYSTEM_ID, 
    CPPROP_SOURCE_SYS_ORIGIN, 
    CPPROP_PARENT_KEY, 
    CASEPH_KEY, 
    CPPTP_KEY, 
    CTP_KEY, 
    CASE_KEY, 
    CASE_START_DATE, 
    CPPROP_VALUE_TEXT, 
    CPPROP_VALUE_DATE, 
    CPPROP_VALUE_NUMBER, 
    CPPROP_VALID_FROM, 
    CPPROP_VALID_TO,
    CPPROP_CURRENT_FLAG, 
    CPPROP_DELETED_FLAG, 
    CPPROP_INSERTED_DATETIME, 
    CPPROP_INSERT_PROCESS_KEY, 
    CPPROP_UPDATED_DATETIME, 
    CPPROP_UPDATE_PROCESS_KEY, 
    CPPRV_KEY)
  select CPPROP_KEY, 
    CPPROP_SOURCE_ID, 
    CPPROP_SOURCE_SYSTEM_ID, 
    CPPROP_SOURCE_SYS_ORIGIN, 
    CPPROP_PARENT_KEY, 
    CASEPH_KEY, 
    CPPTP_KEY, 
    CTP_KEY, 
    CASE_KEY, 
    CASE_START_DATE, 
    CPPROP_VALUE_TEXT, 
    CPPROP_VALUE_DATE, 
    CPPROP_VALUE_NUMBER, 
    to_date(p_load_date,'yyyy-MM-dd') as CPPROP_VALID_FROM, 
    to_date('30000101','yyyyMMdd') as CPPROP_VALID_TO,
    'Y' as CPPROP_CURRENT_FLAG, 
    tech_del_flg as CPPROP_DELETED_FLAG, 
    CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME, 
    p_process_key as CPPROP_INSERT_PROCESS_KEY, 
    CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME, 
    p_process_key as CPPROP_UPDATE_PROCESS_KEY, 
    CPPRV_KEY
    from gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF
  where tech_new_rec = 'N';

  insert into
    gap_catalog.ads_owner.CASE_PHASE_PROPERTIES (
      CPPROP_KEY,
      CPPROP_SOURCE_ID,
      CPPROP_SOURCE_SYSTEM_ID,
      CPPROP_SOURCE_SYS_ORIGIN,
      CPPROP_PARENT_KEY,
      CASEPH_KEY,
      CPPTP_KEY,
      CTP_KEY,
      CASE_KEY,
      CASE_START_DATE,
      CPPROP_VALUE_TEXT,
      CPPROP_VALUE_DATE,
      CPPROP_VALUE_NUMBER,
      CPPROP_VALID_FROM,
      CPPROP_VALID_TO,
      CPPROP_CURRENT_FLAG,
      CPPROP_DELETED_FLAG,
      CPPROP_INSERTED_DATETIME,
      CPPROP_INSERT_PROCESS_KEY,
      CPPROP_UPDATED_DATETIME,
      CPPROP_UPDATE_PROCESS_KEY,
      CPPRV_KEY
    )
  select
    xxhash64(uuid()) AS CPPROP_KEY,
    CPPROP_SOURCE_ID,
    CPPROP_SOURCE_SYSTEM_ID,
    CPPROP_SOURCE_SYS_ORIGIN,
    CPPROP_PARENT_KEY,
    CASEPH_KEY,
    CPPTP_KEY,
    CTP_KEY,
    CASE_KEY,
    CASE_START_DATE,
    CPPROP_VALUE_TEXT,
    CPPROP_VALUE_DATE,
    CPPROP_VALUE_NUMBER,
    to_date(p_load_date, 'yyyy-MM-dd') as CPPROP_VALID_FROM,
    to_date('30000101', 'yyyyMMdd') as CPPROP_VALID_TO,
    'Y' as CPPROP_CURRENT_FLAG,
    tech_del_flg as CPPROP_DELETED_FLAG,
    CURRENT_TIMESTAMP() as CPPROP_INSERTED_DATETIME,
    p_process_key as CPPROP_INSERT_PROCESS_KEY,
    CURRENT_TIMESTAMP() as CPPROP_UPDATED_DATETIME,
    p_process_key as CPPROP_UPDATE_PROCESS_KEY,
    CPPRV_KEY
  from
    gap_catalog.ads_owner.DIFF_ADS_SMA_CASE_PHASE_PROPERTIES_HUMANTASK_ADS_MAP_SCD_DIFF
  where
    tech_new_rec = 'Y';

END;